# Data Cleaning Process

In [2]:
import pandas as pd
from scipy.spatial import cKDTree
import numpy as np
import warnings

warnings.filterwarnings('ignore')

## 1. Calculate Branch Share, Market Share, and Network Equity

In [3]:
def caculate_networkequity(df, bank_name = "Bank of America, National Association", max_distances = 10):
    df = df[
        df['SIMS_LATITUDE'].notna() &
        df['SIMS_LONGITUDE'].notna()
    ]
    
    # Step 1: Filter BoA and non-BoA branches
    boa_df = df.loc[df["NAMEFULL"] == bank_name].copy()
    non_boa_df = df.loc[~(df["NAMEFULL"] == bank_name)].copy()

    # Prepare coordinates for BoA and non-BoA branches
    boa_coords = boa_df[['SIMS_LATITUDE', 'SIMS_LONGITUDE']].to_numpy()
    non_boa_coords = non_boa_df[['SIMS_LATITUDE', 'SIMS_LONGITUDE']].to_numpy()

    # Approximate conversion factor for lat/lon degrees to miles
    # 1 degree latitude: 69 miles, 1 degree longitude: 54.6 miles at 40° latitude (rough average)
    lat_to_miles = 69.0
    lon_to_miles = 54.6

    # Convert lat/lon to approximate miles for KDTree
    boa_coords_miles = np.column_stack((boa_coords[:, 0] * lat_to_miles, boa_coords[:, 1] * lon_to_miles))
    non_boa_coords_miles = np.column_stack((non_boa_coords[:, 0] * lat_to_miles, non_boa_coords[:, 1] * lon_to_miles))

    # Build KDTree from non-BoA coordinates
    tree = cKDTree(non_boa_coords_miles)

    # Query how many non-BoA branches are within 'max_distances' miles for each BoA branch
    radius = max_distances
    non_boa_nearby_counts = tree.query_ball_point(boa_coords_miles, r=radius)
    boa_df['Total Nearby Competitor Branches'] = [len(c) for c in non_boa_nearby_counts]

    # Build KDTree from BoA coordinates only (converted to miles)
    boa_tree = cKDTree(boa_coords_miles)

    # Query how many BoA branches are within 'max_distances' miles for each BoA branch
    # Subtract 1 to exclude the branch itself
    boa_nearby_counts = boa_tree.query_ball_point(boa_coords_miles, r=radius)
    boa_df['Nearby BoA Branches'] = [len(c) - 1 for c in boa_nearby_counts]

    # Convert deposits column to numeric
    boa_df['DEPSUMBR'] = pd.to_numeric(boa_df['DEPSUMBR'], errors='coerce')
    non_boa_df['DEPSUMBR'] = pd.to_numeric(non_boa_df['DEPSUMBR'], errors='coerce')

    boa_df['Total Nearby BoA Deposits'] = [
        boa_df.iloc[[j for j in inds if j != i]]['DEPSUMBR'].sum()
        for i, inds in enumerate(boa_nearby_counts)
    ]

    boa_df['Total Nearby Competitor Deposits'] = [
        non_boa_df.iloc[inds]['DEPSUMBR'].sum()
        for inds in non_boa_nearby_counts
    ]

    # Branch Share: (BoA branches) / (BoA branches + Non BoA branches)
    boa_df['Branch Share'] = (boa_df['Nearby BoA Branches'] + 1) / (
        boa_df['Nearby BoA Branches'] + boa_df['Total Nearby Competitor Branches'] + 1
    )

    # Market share: (nearby BoA deposits + this branch's deposit) / (total nearby BoA + competitor deposits + this branch's deposit)
    boa_df['Market Share'] = (
        boa_df['Total Nearby BoA Deposits'] + boa_df['DEPSUMBR']
    ) / (
        boa_df['Total Nearby BoA Deposits'] + boa_df['Total Nearby Competitor Deposits'] + boa_df['DEPSUMBR']
    )

    boa_df["Network Equity"] = boa_df["Market Share"] / boa_df["Branch Share"]

    return boa_df

In [4]:
BANKNAME = "Bank of America, National Association"
MAXDISTANCES = 10

dfs_filtered = []

for year in list(range(2014, 2025)):
    data_path = f"./data/metadata//FDIC_data/SOD{year}.csv"
    df = pd.read_csv(data_path)
    boa_df = caculate_networkequity(df, BANKNAME, MAXDISTANCES)
    dfs_filtered.append(boa_df)

## 2. Merge all data into one Dataframe

In [14]:
# Combine all years
df_all = pd.concat(dfs_filtered, ignore_index=True)

# Sort by branch and year
df_all_sorted = df_all.sort_values(by=['BRNUM', 'YEAR'])

# Calculate DEPSUMBR change
df_all_sorted['DEPSUMBR_PREV'] = df_all_sorted.groupby('BRNUM')['DEPSUMBR'].shift(1)
df_all_sorted['DEPSUMBR_CHANGE'] = (
    (df_all_sorted['DEPSUMBR'] - df_all_sorted['DEPSUMBR_PREV']) / df_all_sorted['DEPSUMBR_PREV']
)
df_all_sorted['DEPSUMBR_CHANGE'] = df_all_sorted['DEPSUMBR_CHANGE'].fillna(0)

# Get latest row per branch 
df_latest = df_all_sorted.drop_duplicates(subset='BRNUM', keep='last')
df_latest['closed'] = (df_latest['YEAR'] < 2024).astype(int)

## 3. Merge with income data

In [32]:
#income data
income2024 = pd.read_csv('./data/metadata//Income_Data/ACSST5Y2023.S1901-Data.csv')
income2023 = pd.read_csv('./data/metadata//Income_Data/ACSST5Y2022.S1901-Data.csv')
income2022 = pd.read_csv('./data/metadata//Income_Data/ACSST5Y2021.S1901-Data.csv')
income2021 = pd.read_csv('./data/metadata//Income_Data/ACSST5Y2020.S1901-Data.csv')
income2020 = pd.read_csv('./data/metadata//Income_Data/ACSST5Y2019.S1901-Data.csv')
income2019 = pd.read_csv('./data/metadata//Income_Data/ACSST5Y2018.S1901-Data.csv')
income2018 = pd.read_csv('./data/metadata//Income_Data/ACSST5Y2017.S1901-Data.csv')
income2017 = pd.read_csv('./data/metadata//Income_Data/ACSST5Y2016.S1901-Data.csv')
income2016 = pd.read_csv('./data/metadata//Income_Data/ACSST5Y2015.S1901-Data.csv')
income2015 = pd.read_csv('./data/metadata//Income_Data/ACSST5Y2014.S1901-Data.csv')
income2014 = pd.read_csv('./data/metadata//Income_Data/ACSST5Y2013.S1901-Data.csv')

income_dfs = [
    income2014, income2015, income2016, income2017, income2018,
    income2019, income2020, income2021, income2022, income2023, income2024
]
income_years = list(range(2014, 2025)) 
df_latest['ZIPBR'] = df_latest['ZIPBR'].astype(str).str.zfill(5) 
df_latest = df_latest.drop(columns=['ZIP'], errors='ignore')
cleaned_income_dfs = {}

for df, year in zip(income_dfs, income_years):
    df_clean = df.iloc[1:].copy()
    df_clean = df_clean.rename(columns={'NAME': 'ZIP'})
    df_clean['ZIP'] = df_clean['ZIP'].str.replace("ZCTA5 ", "").str.zfill(5)
    df_clean['YEAR'] = year
    cleaned_income_dfs[year] = df_clean

merged_list = []

for year in df_latest['YEAR'].unique():
    df_year = df_latest[df_latest['YEAR'] == year]
    income_df = cleaned_income_dfs.get(year)

    if income_df is not None:
        merged = df_year.merge(income_df, left_on=['ZIPBR', 'YEAR'], right_on=['ZIP', 'YEAR'], how='left')
        merged_list.append(merged)
    else:
        merged_list.append(df_year)

df_income_merged = pd.concat(merged_list, ignore_index=True)
na_rows = df_income_merged[df_income_merged['ZIP'].isna()]
df_income_merged = df_income_merged[df_income_merged['ZIP'].notna()] 

cols_to_keep = [
    'S1901_C01_012E',
    'S1901_C01_013E'
]

# Drop all columns that start with 'S1901' but are NOT in cols_to_keep
df_income_merged = df_income_merged.drop(
    columns=[col for col in df_income_merged.columns
             if col.startswith('S1901') and col not in cols_to_keep]
)

In [33]:
df_income_merged['S1901_C01_013E'] = pd.to_numeric(
    df_income_merged['S1901_C01_013E'], errors='coerce'
) 
df_income_merged['S1901_C01_012E'] = pd.to_numeric(
    df_income_merged['S1901_C01_012E'], errors='coerce'
) 

## 4. Rename columns and drop unecessary ones

In [34]:
df_income_merged = df_income_merged.drop(['UNINUMBR', 'ZIPBR', 'GEO_ID', 'Unnamed: 130','DEPSUMBR_PREV'], axis=1)

In [37]:
cols = {
    'YEAR' : 'year', 
    'CERT' : 'FDIC_cert', 
    'NAMEFULL': "bank_name", 
    'ADDRESBR': "adress", 
    'BRNUM': "branch_id", 
    'STALPBR': "state_abb",
    'BRSERTYP': "branch_service_type", 
    'CITYBR': "city", 
    'CNTYNAMB': "county", 
    'CSABR': "combined_stat_area", 
    'CSANAMBR': "combined_stat_area_name", 
    'DEPSUMBR': "deposits",
    'METROBR': "is_metropolitan", 
    'MICROBR': "is_micropolitan", 
    'MSABR': "metro_stat_area", 
    'NAMEBR': "branch_name", 
    'SIMS_ESTABLISHED_DATE': "established_date",
    'SIMS_LATITUDE': "latitude", 
    'SIMS_LONGITUDE': "longitude", 
    'STCNTYBR': "state_county_num", 
    'STNAMEBR': "state_name", 
    'Total Nearby Competitor Branches': "total_nearby_competitor_branches", 
    'Nearby BoA Branches': "nearby_boa_branches",
    'Total Nearby BoA Deposits': "total_nearby_boa_deposits", 
    'Total Nearby Competitor Deposits': "total_nearby_competitor_deposits",
    'Branch Share': "branch_share", 
    'Market Share': "market_share", 
    'Network Equity': "network_equity", 
    'ZIP': "zipcode", 
    'DEPSUMBR_CHANGE': "deposit_change",
    'S1901_C01_012E': "estimate_household_median_income", 
    'S1901_C01_013E': "estimate_household_mean_income"
}

df_final= df_income_merged.rename(columns=cols)

In [38]:
# store final data
df_final.to_csv('data/cleaned_data/cleaned_data.csv')